In [23]:
import pandas as pd

df = pd.read_csv('../data/ai_vs_human_code_dataset.csv')
# code id varmaan turha (?) + mitenköhän tuo code_snippets variable hyödynnetään?
# olisko mahdollista tehdä malli, jossa syötettäisiin itse koodinpätkiä ja malli predictaisi onko human vai ai?
# logistista regressiota olisi mahdollista käyttää kyseiseen toteutukseen. Samalla voitaisiin analysoida (coefficients) ja tutkia mitkä ovat tärkeitä variableja siinä että onko koodi ai vai human.

df


,code_id,code_snippet,code_length,num_functions,num_classes,num_variables,num_comments,comment_density,num_imports,use_of_recursion,use_of_lambda,num_loops,complexity_score,execution_time,ai_generated
0,04bd1e86,import random\ndef random_numbers(n): return [...,86,1,0,0,0,0.0,1,0,0,1,2,0.60,1
1,03020544,"x = lambda a, b: a + b\nprint(x(3, 4))",37,0,0,1,0,0.0,0,0,1,0,0,1.83,1
2,a6a08557,"def fib(n): a, b = 0, 1\n for _ in range(n): a...",69,1,0,2,0,0.0,0,0,0,1,2,0.51,1
3,edce6fd4,def factorial(n): return 1 if n == 0 else n * ...,62,1,0,0,0,0.0,0,1,0,0,3,10.93,1
4,03020544,"x = lambda a, b: a + b\nprint(x(3, 4))",37,0,0,1,0,0.0,0,0,1,0,0,3.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,a8ee5b6a,"import statistics\nvalues = [10, 20, 30]\nprin...",70,0,0,1,0,0.0,1,0,0,0,0,1.47,0
1496,7147f334,"def add_numbers(a, b):\n return a + b\nprin...",64,1,0,0,0,0.0,0,0,0,0,1,1.14,0
1497,0e7f4f30,"def fibonacci(n):\n sequence = [0, 1]\n ...",142,1,0,1,0,0.0,0,0,0,1,2,2.22,0
1498,b48fc10d,import random\nrandom.seed(42)\ndef gen_random...,102,1,0,0,0,0.0,1,0,0,1,2,2.30,0
